# Explore SUIT Space and Cerebellar Atlases

This notebook provides an interactive exploration of the SUIT cerebellar space,
the lobular parcellation, deep cerebellar nuclei, and the flatmap projection.

Run this after completing Step 0 (data acquisition).

In [ ]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from nilearn import plotting
import sys
sys.path.insert(0, '..')
from src.utils import ATLAS_DIR, DATA_RAW, PROJECT_ROOT, load_nifti

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

## 1. Load and Inspect the SUIT Template

In [ ]:
# Locate the SUIT template from SUITPy
try:
    import SUITPy
    import os
    suit_dir = os.path.dirname(SUITPy.__file__)
    print(f"SUITPy installed at: {suit_dir}")
    print(f"Contents: {os.listdir(suit_dir)}")
except ImportError:
    print("SUITPy not installed. Run: pip install git+https://github.com/DiedrichsenLab/SUITPy.git")

In [ ]:
# Load the SUIT template
suit_template_path = os.path.join(suit_dir, 'data', 'SUIT.nii')
if os.path.exists(suit_template_path):
    template = nib.load(suit_template_path)
    print(f"Template shape: {template.shape}")
    print(f"Voxel size: {template.header.get_zooms()}")
    print(f"Affine:\n{template.affine}")
else:
    print(f"Template not found at {suit_template_path}")
    print("Check SUITPy data directory for the correct filename.")

In [ ]:
# Visualize the SUIT template
plotting.plot_anat(suit_template_path, title='SUIT Template',
                   display_mode='ortho', cut_coords=(0, -60, -30))
plt.show()

## 2. Explore the Lobular Parcellation

In [ ]:
# Load the SUIT anatomical parcellation
atlas_path = ATLAS_DIR / 'atl-Anatom_space-SUIT_dseg.nii'
if atlas_path.exists():
    atlas = nib.load(str(atlas_path))
    atlas_data = atlas.get_fdata().astype(int)
    unique_labels = np.unique(atlas_data)
    print(f"Atlas shape: {atlas.shape}")
    print(f"Unique labels ({len(unique_labels)}): {unique_labels}")
    print(f"Non-zero voxels: {np.count_nonzero(atlas_data)}")
else:
    print(f"Atlas not found at {atlas_path}")
    print("Run: python -m src.download")

In [ ]:
# Plot the lobular parcellation overlaid on the template
if atlas_path.exists():
    plotting.plot_roi(str(atlas_path), bg_img=suit_template_path,
                      title='SUIT Lobular Parcellation',
                      display_mode='ortho', cut_coords=(0, -60, -30))
    plt.show()

## 3. Explore the Deep Cerebellar Nuclei

In [ ]:
# Look for nuclei probability maps in the atlas directory
import glob
nuclei_files = sorted(glob.glob(str(ATLAS_DIR / '*nuclei*'))) + \
               sorted(glob.glob(str(ATLAS_DIR / '*Dentate*'))) + \
               sorted(glob.glob(str(ATLAS_DIR / '*Fastigial*'))) + \
               sorted(glob.glob(str(ATLAS_DIR / '*Interpos*')))
print("Nuclei-related files found:")
for f in nuclei_files:
    print(f"  {os.path.basename(f)}")

if not nuclei_files:
    print("No nuclei files found. Searching more broadly...")
    all_files = sorted(glob.glob(str(ATLAS_DIR / '**' / '*'), recursive=True))
    for f in all_files[:30]:
        print(f"  {os.path.relpath(f, ATLAS_DIR)}")

## 4. Explore the Flatmap

In [ ]:
# Try SUITPy flatmap projection
try:
    import SUITPy as suit
    
    # Check available functions
    print("SUITPy flatmap functions:")
    print([x for x in dir(suit.flatmap) if not x.startswith('_')])
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# Project the lobular atlas onto the flatmap
try:
    if atlas_path.exists():
        surf_data = suit.flatmap.vol_to_surf(str(atlas_path))
        fig = suit.flatmap.plot(
            surf_data,
            render='matplotlib',
            cmap='tab20',
            new_figure=True
        )
        plt.title('Lobular Parcellation on SUIT Flatmap')
        plt.show()
except Exception as e:
    print(f"Flatmap projection failed: {e}")
    print("Check SUITPy documentation for the correct API.")

## 5. Understand the Coordinate System

SUIT space is centered near the midline of the cerebellum:
- **x ≈ 0** is the midline (vermis)
- **x < 0** is the left hemisphere
- **x > 0** is the right hemisphere

This medial-lateral axis is critical for the zone classification (vermis / paravermis / lateral).

In [ ]:
# Examine the x-coordinate distribution of cerebellar cortex voxels
if atlas_path.exists():
    from src.utils import get_mm_coordinate_grid
    
    mm_grid = get_mm_coordinate_grid(atlas_data.shape, atlas.affine)
    cortex_mask = atlas_data > 0
    x_coords = mm_grid[cortex_mask, 0]
    
    fig, ax = plt.subplots(1, 1, figsize=(10, 4))
    ax.hist(x_coords, bins=100, edgecolor='black', alpha=0.7)
    ax.axvline(0, color='red', linestyle='--', label='Midline')
    ax.axvline(-5, color='orange', linestyle=':', label='Vermis boundary (5mm)')
    ax.axvline(5, color='orange', linestyle=':')
    ax.axvline(-15, color='green', linestyle=':', label='Paravermis boundary (15mm)')
    ax.axvline(15, color='green', linestyle=':')
    ax.set_xlabel('x-coordinate (mm)')
    ax.set_ylabel('Number of cortex voxels')
    ax.set_title('Medial-Lateral Distribution of Cerebellar Cortex Voxels')
    ax.legend()
    plt.tight_layout()
    plt.show()

## 6. Zone Classification Preview

In [ ]:
# Visualize the zone classification
if atlas_path.exists():
    from src.utils import classify_zones
    
    x_mm = mm_grid[..., 0]
    zones = classify_zones(x_mm)
    
    # Create a zone map: 1=vermis, 2=paravermis, 3=lateral
    zone_map = np.zeros_like(atlas_data, dtype=float)
    zone_map[cortex_mask] = (
        1 * zones['vermis'][cortex_mask] +
        2 * zones['paravermis'][cortex_mask] +
        3 * zones['lateral'][cortex_mask]
    )
    
    # Plot
    zone_img = nib.Nifti1Image(zone_map, atlas.affine)
    plotting.plot_roi(zone_img, bg_img=suit_template_path,
                      title='Zone Classification (1=vermis, 2=paravermis, 3=lateral)',
                      display_mode='ortho', cut_coords=(0, -60, -30),
                      cmap='Set1')
    plt.show()